In [1]:
import pandas
import ee  # Google Earth Engine API
import folium
import functools
import numpy as np

In [2]:

ee.Initialize()

## Load data from the CSV


In [3]:
csv_path = 'Final_metadata_05312019.csv'

# Read the relevant columns from the CSV into a dataframe
# We will use the coordinates to get more data about each sample
samples = pandas.read_csv(csv_path, usecols=['MatchName', 'Longitude', 'Latitude'])

# Display the shape of the data we read in and its first few rows
print("Data shape:", samples.shape)
print(samples.head())

# Define a Feature for each 100-meter-radius sample area
sample_areas = []
for sample in samples.itertuples():
    # Store the important data as properties of the feature
    sample_areas.append(
        ee.Feature(ee.Geometry.Point(sample.Longitude, sample.Latitude).buffer(100))
        .set('MatchName', sample.MatchName)
        .set('Longitude', sample.Longitude)
        .set('Latitude', sample.Latitude))

# Define a FeatureCollection containing all the sample areas
sample_areas = ee.FeatureCollection(sample_areas)

Data shape: (278, 3)
  MatchName   Longitude   Latitude
0   K0024A2 -118.567312  34.083974
1   K0024B2 -118.570642  34.084767
2   K0024C1 -118.551882  34.055978
3   K0026A1 -117.229718  32.853963
4   K0026B1 -117.232750  32.849619


First define some utility functions.

Earth Engine coordinates are given in the order (longitude, latitude), while Folium uses them in the order (latitude, longitude), so swapping the order will sometimes be necessary.

In [4]:

def to_number(key: str):
    """
    Return a lambda function that converts a feature's property value from a string to a number.
    To be used with map().
    
    param key: the name of the feature property to convert
    """
    return lambda feature: feature.set({key: ee.Number.parse(feature.get(key))})

## Define dataset properties

In [5]:
class Dataset:
    def __init__(self, path: str, property: str, name: str):
        """
        Represent a Google Earth Engine dataset and the information we need to use it.
        
        path: The identifier of the dataset for EE e.g. 'EPA/Ecoregions/2013/L3'
        property: Many datasets have multiple properties (for vector datasets) or
            bands (for raster datasets). This is the name of the relevant one.
        name: The column name to display for the data gathered from this dataset.
        """
        self.data = ee.FeatureCollection(path).map(to_number(key='us_l3code'))
        self.property = property
        self.name = name
epa_ecoregions = Dataset('EPA/Ecoregions/2013/L3', 'us_l3name', 'ecoregion')


## Get data about a sample location


In [6]:
# Note: This function is an argument to map(). Arguments to map() cannot print anything
# or call getInfo(). Doing so results in an EEException: ValueNode empty
# source: https://gis.stackexchange.com/questions/345598/mapping-simple-function-to-print-date-and-time-stamp-on-google-earth-engine-pyth
def get_sample_area_data(sample_area, dataset: Dataset):
    
    
    overlaps = dataset.data.filterBounds(sample_area.geometry()).map(
        lambda feature: feature.intersection(sample_area.geometry())
    )
    count = overlaps.size()
    
    val = ee.Algorithms.If(
        count.eq(1),
        sample_area.set(dataset.name, overlaps.first().get(dataset.property)),
        ee.Algorithms.If(
            count.eq(0),
            sample_area.set(dataset.name, get_nearest_feature(sample_area, dataset)),
            sample_area.set(dataset.name, get_predominant_feature(overlaps, dataset.property))
        )
    )
    return val

def get_nearest_feature(sample_area: ee.Feature, dataset: Dataset) -> str:
    """
    To be used when the sample area doesn't overlap the dataset at all.
    Get the dataset feature that is nearest to the sample area, and
    return the value of its dataset.property.
    """
    
    # Define a filter to get all dataset features within 10000 meters of the sample area
    spatialFilter = ee.Filter.withinDistance(
        distance=10000,
        leftField='.geo',
        rightField='.geo',
        maxError=10
    )
    
    # Define a join that will return only the 'best' (nearest) match
    saveBestJoin = ee.Join.saveBest(
      matchKey='closestFeature',
      measureKey='distance'
    )
    # Apply the join, using the distance filter to define match quality
    # Get the only feature in the resulting FeatureCollection
    result = ee.Feature(saveBestJoin.apply(
        ee.FeatureCollection(sample_area),
        dataset.data,
        spatialFilter
    ).first())
    
    # Return the value of the relevant property of the closest dataset feature
    return ee.Feature(result.get('closestFeature')).get(dataset.property)

    
def get_predominant_feature(overlaps: ee.FeatureCollection, property: str) -> str:
    """
    To be used when the sample area overlaps more than one dataset feature.
    Return the value of 'property' for the largest overlap.
    """
    # Add 'area' as a property to each feature. This is the area in square meters 
    # of the intersection of the ecoregion feature and the sample area.
    overlaps = overlaps.map(
        lambda feature: feature.set({'area': feature.geometry().area()}))
    
    # Find the maximum area among all the overlaps
    max_area = overlaps.aggregate_max('area')

    # Return the value of 'property' for the overlap with the largest area
    return overlaps.filter(ee.Filter.gte('area', max_area)).first().get(property)


    

## Mapping
Next we define a mapping class and some methods to help visualize the data. This will help with understanding what we're doing, and also make it easy to visually verify that the results make sense. The JavaScript version of the Earth Engine API provides a Map class that makes it easy to do this, but the Python API doesn't have that feature. I'm implementing it using Folium, a Python library for creating Leaflet Javascript maps, as recommended here: https://developers.google.com/earth-engine/python_install-colab.html#interactive_map

These two methods are going to be added as custom folium map methods. This will make it easy to display vector and raster Earth Engine objects to a map.

In [7]:
def add_raster_layer(self, ee_image_object, vis_params, name):
    """Display an EE image (raster) on a folium map"""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Map Data © Google Earth Engine",
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

def add_vector_layer(self, coords, name):
    """Display an EE geometry (vector) on a folium map"""
    # Reverse the order of the from (lng, lat) to (lat, lng)
    coords = np.array([np.flip(i) for i in coords])
    
    # Add the coordinates as a polygon layer in the map
    folium.vector_layers.Polygon(
        locations=coords,
        name=name,
        color='red',
        overlay=True,
        control=True
    ).add_to(self)

Next we define the Map class, which makes use of the above methods to draw multiple datasets onto a folium map and display it.

In [8]:
class Map:
    
    def __init__(self):
        """Initialize a custom folium map"""
        # Add EE drawing methods to folium.
        folium.Map.add_raster_layer = add_raster_layer
        folium.Map.add_vector_layer = add_vector_layer
        self.map = folium.Map(location=[39, -98], zoom_start=3, height=500)

    def add(self, dataset):
        """Add layers to an interactive map using folium"""
        # Set visualization parameters.
        vis_params = {
          'min': 0,
          'max': 120,
          'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

        # Add the dataset as a layer to the map
        # If the data is provided as an array, we want to display it as a polygon
        if isinstance(dataset, np.ndarray):
            self.map.add_vector_layer(dataset, 'sample area')
            return

        # If the dataset is provided as a Collection, we want to display it as imagery
        if isinstance(dataset, ee.FeatureCollection):
            dataset = dataset.reduceToImage(properties=['us_l3code'], reducer=ee.Reducer.first())

        self.map.add_raster_layer(dataset.updateMask(dataset.gt(0)), vis_params, 'ecoregions')
        
    def feature_collection_to_coords(fc: ee.FeatureCollection) -> np.array:
        """
        Return a list of coordinate lists representing the geometries 
        of the features in the FeatureCollection.
        """
        coords = np.array(fc.iterate(
            lambda item, l: ee.List(l).add(item.geometry().coordinates()), 
            ee.List([])).getInfo())
        print(coords.shape)
        return coords

    def display(self):
            self.map.add_child(folium.LayerControl())  # Add a layer control panel to the map
            display(self.map)   # Display the map


In [9]:

np.set_printoptions(precision=20)
# r = ee.Feature(sample_areas.filterMetadata('MatchName', 'starts_with', 'K0194A1').first())
# r = ee.FeatureCollection(r)

r = sample_areas.map(lambda feature: get_sample_area_data(feature, epa_ecoregions))

coords = Map.feature_collection_to_coords(r)

# Display the map
map2 = Map()
for c in coords:
    map2.add(c)
map2.add(epa_ecoregions.data)

map2.display()

task = ee.batch.Export.table.toDrive(collection=r, description='eedata.csv')
task.start()


<class 'ee.featurecollection.FeatureCollection'>
{'type': 'FeatureCollection', 'columns': {'Latitude': 'Float', 'Longitude': 'Float', 'MatchName': 'String', 'ecoregion': 'Object', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-118.56731200000002, 34.08487388659514], [-118.56761548675433, 34.0848380707356], [-118.56789481524308, 34.08473347418612], [-118.56812775042329, 34.08456842306514], [-118.56829575053854, 34.08435605574865], [-118.56838544307917, 34.084113276959684], [-118.56838968914022, 34.083859412063575], [-118.56830815148479, 34.08361466871466], [-118.56814732115384, 34.08339852832429], [-118.56792000057874, 34.083228195390184], [-118.56764428440235, 34.083117228104996], [-118.56734211916915, 34.083074459220526], [-118.56703755652441, 34.08310329303686], [-118.56675483892437, 34.0832014344523], [-118.56651647016427, 34.08336107163152], [-118.56634142424046, 34.083569497758674], [-118.56624363506933, 34.0838101224

31040.60634797471
(278, 1, 24, 2)
[[[[-118.56731200000002    34.08487388659514 ]
   [-118.56761548675433    34.0848380707356  ]
   [-118.56789481524308    34.08473347418612 ]
   ...
   [-118.56695115990254    34.08482281275847 ]
   [-118.56725178353224    34.084872503568874]
   [-118.56731200000002    34.08487388659514 ]]]


 [[[-118.57064199999999    34.08566688659515 ]
   [-118.57094548959662    34.08563107073561 ]
   [-118.57122482070143    34.08552647418608 ]
   ...
   [-118.57028115652305    34.085615812758455]
   [-118.57058178296828    34.085665503568876]
   [-118.57064199999999    34.08566688659515 ]]]


 [[[-118.55188200000002    34.05687788659516 ]
   [-118.55218538648077    34.056842070736   ]
   [-118.55246462267871    34.05673747418757 ]
   ...
   [-118.55152127912588    34.05682681275903 ]
   [-118.55182180342811    34.0568765035689  ]
   [-118.55188200000002    34.05687788659516 ]]]


 ...


 [[[-118.28106800000002    33.70982188659514 ]
   [-118.28137015487962    33.709